# **Índice de Moran**

## **Librerías y modulos necesarios**

In [1]:
import json
import random
import numpy as np
import pandas as pd

## **Data**

Inicialmente, se hace el cargue de los conjuntos de datos a utilizar.

Para el cálculo de la tasa de mortalidad, es necesario tener los datos sobre que tantas mujeres fallecidas hubieron a lo largo del período por dicha enfermedad.

In [2]:
muertes = pd.read_excel('../data_tasas/MUERTES-CANMAMA-M40-MUN-COL.xlsx')
muertes.head()

,DPMP,DPNOM,ANIO,FALLECIDAS
0,EL ENCANTO,AMAZONAS,2009,0
1,EL ENCANTO,AMAZONAS,2010,0
2,EL ENCANTO,AMAZONAS,2011,0
3,EL ENCANTO,AMAZONAS,2012,0
4,EL ENCANTO,AMAZONAS,2013,0


También, se utilizará la población total de mujeres mayores de 40 años por municipio en ese período de años (2009 - 2023).

In [3]:
poblacion = pd.read_excel('../data_tasas/POB-M40-MUNCOL-2009-2023.xlsx')
poblacion_total = poblacion[['DPNOM', 'DPMP', 'MPIO', 'ANIO', 'TOTAL_MUJERES']]
poblacion_total.head()

,DPNOM,DPMP,MPIO,ANIO,TOTAL_MUJERES
0,AMAZONAS,EL ENCANTO,91263,2009,205
1,AMAZONAS,EL ENCANTO,91263,2010,213
2,AMAZONAS,EL ENCANTO,91263,2011,224
3,AMAZONAS,EL ENCANTO,91263,2012,229
4,AMAZONAS,EL ENCANTO,91263,2013,233


Se lee el DataFrame del archivo de metadatos geográficos de cada municipio.

In [4]:
mdata_mun = pd.read_excel('../data_tasas/metadata_mun_col.xlsx')
mdata_mun.head()

,MPIO,DPMP,DPNOM,GEOMETRY
0,91405,LA CHORRERA,AMAZONAS,"POLYGON ((5000339.8632 1499049.6806000005, 501..."
1,91669,SANTANDER,AMAZONAS,"POLYGON ((5109398.9191 1528841.3461000007, 510..."
2,91460,MIRITÍ-PARANÁ,AMAZONAS,"POLYGON ((5161666.8311 1560407.6130999997, 516..."
3,91430,LA VICTORIA,AMAZONAS,"POLYGON ((5201993.3468 1568778.1232999992, 520..."
4,52560,POTOSÍ,NARIÑO,"POLYGON ((4492613.2148 1651194.4537000004, 449..."


Se carga el archivo que contiene los vecinos municipales de cada municipio.

In [5]:
with open('vecinos_municipales.json', 'r', encoding='utf-8') as f:
    vecinos_mun = json.load(f)

## **Tasa de mortalidad cruda por municipio**

Inicialmente, para calcular la tasa de mortalidad, se agrupan la cantidad de fallecimientos por departamentos.

In [6]:
muertes.groupby(['DPNOM'])['FALLECIDAS'].sum()

DPNOM
AMAZONAS                                                       18
ANTIOQUIA                                                    6102
ARAUCA                                                        169
ARCHIPIÉLAGO DE SAN ANDRÉS Y PROVIDENCIA Y SANTA CATALINA      56
ATLÁNTICO                                                    2928
BOLÍVAR                                                      1664
BOYACÁ                                                        869
CALDAS                                                       1065
CAQUETÁ                                                       240
CASANARE                                                      218
CAUCA                                                         807
CESAR                                                         757
CHOCÓ                                                         132
CUNDINAMARCA                                                 8956
CÓRDOBA                                                      1006
GUAI

Se procede a realizar el cálculo de esta tasa con la siguiente función

In [7]:
def calcular_tasa_mortalidad(muertes, poblacion, año_inicio=2009, año_fin=2023, k=100000):
    """
    Calcula la tasa cruda de mortalidad por municipio y departamento para mujeres mayores de 40 años.

    Parámetros:
    - muertes: DataFrame con columnas ['DPNOM', 'DPMP', 'ANIO', 'FALLECIDAS']
    - poblacion: DataFrame con columnas ['DPNOM', 'DPMP', 'MPIO', 'ANIO', 'TOTAL_MUJERES']
    - año_inicio: Año inicial del periodo (int)
    - año_fin: Año final del periodo (int)
    - k: Factor de estandarización (int, default 100000)

    Retorna:
    - DataFrame con columnas ['MPIO', 'DPNOM', 'DPMP', 'FALLECIDAS', 'TOTAL_MUJERES', 'Tasa_Mortalidad']
    """
    # Filtrado por años
    muertes_filtrado = muertes[(muertes['ANIO'] >= año_inicio) & (muertes['ANIO'] <= año_fin)]
    pob_filtrada = poblacion[(poblacion['ANIO'] >= año_inicio) & (poblacion['ANIO'] <= año_fin)]

    # Agrupación de muertes por departamento y municipio
    muertes_agg = muertes_filtrado.groupby(['DPNOM', 'DPMP'], as_index=False)['FALLECIDAS'].sum()

    # Agrupación de población (incluyendo MPIO para conservarlo)
    pob_agg = pob_filtrada.groupby(['DPNOM', 'DPMP', 'MPIO'], as_index=False)['TOTAL_MUJERES'].sum()

    # Unión de muertes con población (se une por DPNOM y DPMP)
    tasa = pd.merge(muertes_agg, pob_agg, on=['DPNOM', 'DPMP'])

    # Cálculo de la tasa de mortalidad
    tasa['Tasa_Mortalidad'] = (tasa['FALLECIDAS'] / tasa['TOTAL_MUJERES']) * k

    # Reordenar columnas
    columnas = ['MPIO', 'DPNOM', 'DPMP', 'FALLECIDAS', 'TOTAL_MUJERES', 'Tasa_Mortalidad']
    tasa = tasa[columnas]

    return tasa


In [8]:
tasa_mortalidad  = calcular_tasa_mortalidad(muertes, poblacion_total, año_inicio = 2009, año_fin = 2023, k = 100000)
tasa_mortalidad.head()

,MPIO,DPNOM,DPMP,FALLECIDAS,TOTAL_MUJERES,Tasa_Mortalidad
0,91263,AMAZONAS,EL ENCANTO,0,3478,0.000000
1,91405,AMAZONAS,LA CHORRERA,0,3907,0.000000
2,91407,AMAZONAS,LA PEDRERA,1,5283,18.928639
3,91430,AMAZONAS,LA VICTORIA,0,1378,0.000000
4,91001,AMAZONAS,LETICIA,17,80655,21.077429


Evidentemente, hay varios municipios que tienen una tasa de mortalidad igual a 0, dado que no tienen casos de fallecimientos.

Veamos cuales municipios no tienen tasa de mortalidad igual a 0

In [9]:
tasa_mortalidad.query(" Tasa_Mortalidad > 0")

,MPIO,DPNOM,DPMP,FALLECIDAS,TOTAL_MUJERES,Tasa_Mortalidad
2,91407,AMAZONAS,LA PEDRERA,1,5283,18.928639
4,91001,AMAZONAS,LETICIA,17,80655,21.077429
11,5002,ANTIOQUIA,ABEJORRAL,13,64951,20.015088
12,5004,ANTIOQUIA,ABRIAQUÍ,3,7347,40.832993
14,5030,ANTIOQUIA,AMAGÁ,19,91959,20.661382
...,...,...,...,...,...,...
1111,76895,VALLE DEL CAUCA,ZARZAL,32,130564,24.509053
1114,97511,VAUPÉS,PACOA,1,3894,25.680534
1118,99773,VICHADA,CUMARIBO,2,84025,2.380244
1120,99001,VICHADA,PUERTO CARREÑO,10,36190,27.631943


Ahora, veamos el panorama general de la tasa de mortalidad municipal en Colombia por cáncer de mama en mujeres mayores de 40 años en el rango de años de 2009 a 2023

In [10]:
# Resultados: mayor, menor y promedio
mayor = tasa_mortalidad.loc[tasa_mortalidad['Tasa_Mortalidad'].idxmax()]
menor = tasa_mortalidad.loc[tasa_mortalidad['Tasa_Mortalidad'].idxmin()]
promedio = tasa_mortalidad['Tasa_Mortalidad'].mean()

print("\nMayor tasa municipal:", f"{mayor['DPNOM']} - {mayor['DPMP']}", f"{mayor['Tasa_Mortalidad']:.2f}")
print("Menor tasa municipal:", f"{menor['DPNOM']} - {menor['DPMP']}", f"{menor['Tasa_Mortalidad']:.2f}")
print("Tasa promedio municipal:", f"{promedio:.2f}")


Mayor tasa municipal: SANTANDER - BUCARAMANGA 67.22
Menor tasa municipal: AMAZONAS - EL ENCANTO 0.00
Tasa promedio municipal: 15.40


## **Análisis de Autocorrelación Espacial con el Índice de Moran**

Para poder realizar el cálculo del índice de moran, inicialmente concatenamos los dataframe que contienen la tasa de mortalidad junto con los datos espaciales de cada municipio

In [11]:
tasa_mortalidad_geo = pd.merge(tasa_mortalidad, mdata_mun[['MPIO', 'GEOMETRY']], on='MPIO', how='left')
tasa_mortalidad_geo.head()

,MPIO,DPNOM,DPMP,FALLECIDAS,TOTAL_MUJERES,Tasa_Mortalidad,GEOMETRY
0,91263,AMAZONAS,EL ENCANTO,0,3478,0.000000,"POLYGON ((5000382.5009 1391996.4739999995, 503..."
1,91405,AMAZONAS,LA CHORRERA,0,3907,0.000000,"POLYGON ((5000339.8632 1499049.6806000005, 501..."
2,91407,AMAZONAS,LA PEDRERA,1,5283,18.928639,"POLYGON ((5305430.1299 1510105.1340999994, 530..."
3,91430,AMAZONAS,LA VICTORIA,0,1378,0.000000,"POLYGON ((5201993.3468 1568778.1232999992, 520..."
4,91001,AMAZONAS,LETICIA,17,80655,21.077429,"POLYGON ((5365098.4831 1225751.6786000002, 536..."



Para evaluar la existencia de autocorrelación espacial en las tasas crudas de los municipios, se utilizará el **índice de Moran (I)**, definido como:

$$
I = \frac{n \sum_{i=1}^{n} \sum_{j=1}^{n} w_{ij}(z_i - \bar{z})(z_j - \bar{z})}{S_0 \sum_{i=1}^{n} (z_i - \bar{z})^2}
$$

donde:

- $z_i$: Tasa cruda del municipio $i$  
- $\bar{z}$: Media de las tasas crudas  
- $w_{ij}$: Elemento de la matriz de pesos espaciales, definido como:

$$
w_{ij} =
\begin{cases}
1 & \text{si los municipios } i \text{ y } j \text{ comparten frontera} \\
0 & \text{en cualquier otro caso}
\end{cases}
$$

- $S_0 = \sum_{i=1}^{n} \sum_{j=1}^{n} w_{ij}$: Suma total de los pesos espaciales




In [12]:
tasa_mortalidad['MPIO'] = tasa_mortalidad['MPIO'].astype(str)
# Diccionario de tasas: {mpio: tasa}
tasas = dict(zip(tasa_mortalidad['MPIO'], tasa_mortalidad['Tasa_Mortalidad']))
# Media de la tasa
z_bar = np.mean(list(tasas.values()))


In [13]:
numerador = 0
for i in tasas:
    for j in vecinos_mun.get(i, []):
        if j in tasas:
            numerador += (tasas[i] - z_bar) * (tasas[j] - z_bar)


In [14]:
denominador = sum((zi - z_bar)**2 for zi in tasas.values())

In [15]:
S0 = sum(len([j for j in vecinos_mun.get(i, []) if j in tasas]) for i in tasas)

In [16]:
n = len(tasas)
I = (n / S0) * (numerador / denominador)
print("Índice de Moran I:", I)

Índice de Moran I: 0.14309674177856324


El valor del índice de Moran obtenido fue **I = 0.143**, lo que indica una **autocorrelación espacial positiva moderada** en las tasas de cáncer de mama en mujeres mayores de 40 años en los municipios de Colombia entre los años 2009 y 2023. Este resultado sugiere que **los municipios con tasas similares (ya sean altas o bajas) tienden a agruparse geográficamente**, es decir, no están distribuidos de manera aleatoria en el territorio. En términos prácticos, podría haber **zonas específicas del país donde se concentra una mayor carga de la enfermedad**, lo cual es relevante para orientar estrategias de prevención, diagnóstico temprano y asignación de recursos en salud pública. La identificación de estos patrones espaciales permite una **planificación más focalizada y efectiva** en los territorios que presentan mayor riesgo o vulnerabilidad frente al cáncer de mama.


### **P-valor índice de Moran**

#### **Hipótesis**
- $H_0$: No hay correlación espacial (autocorrelación espacial nula)  
- $H_1$: Hay correlación espacial positiva


In [17]:
def calcular_moran_con_pvalor(muertes, poblacion_total, año_inicio, año_fin, k=100000, permutaciones=999, semilla=42):
    # Calcular tasa de mortalidad
    tasa_mortalidad = calcular_tasa_mortalidad(muertes, poblacion_total, año_inicio=año_inicio, año_fin=año_fin, k=k)
    tasa_mortalidad['MPIO'] = tasa_mortalidad['MPIO'].astype(str)

    # Diccionario de tasas
    tasas = dict(zip(tasa_mortalidad['MPIO'], tasa_mortalidad['Tasa_Mortalidad']))

    # Leer vecinos
    with open('vecinos_municipales.json', 'r', encoding='utf-8') as f:
        vecinos_mun = json.load(f)

    # Tasa media
    z_bar = np.mean(list(tasas.values()))

    # Índice de Moran observado
    def calcular_I(tasas_dict):
        z_bar_local = np.mean(list(tasas_dict.values()))
        numerador = 0
        for i in tasas_dict:
            for j in vecinos_mun.get(i, []):
                if j in tasas_dict:
                    numerador += (tasas_dict[i] - z_bar_local) * (tasas_dict[j] - z_bar_local)
        denominador = sum((zi - z_bar_local) ** 2 for zi in tasas_dict.values())
        S0 = sum(len([j for j in vecinos_mun.get(i, []) if j in tasas_dict]) for i in tasas_dict)
        n = len(tasas_dict)
        return (n / S0) * (numerador / denominador)

    I_observado = calcular_I(tasas)

    # Permutaciones
    random.seed(semilla)
    valores_I = []
    tasas_lista = list(tasas.values())
    llaves = list(tasas.keys())
    for _ in range(permutaciones):
        tasas_perm = dict(zip(llaves, random.sample(tasas_lista, len(tasas_lista))))
        I_perm = calcular_I(tasas_perm)
        valores_I.append(I_perm)

    # p-valor empírico
    extremos = sum(1 for val in valores_I if abs(val) >= abs(I_observado))
    p_valor = (extremos + 1) / (permutaciones + 1)  # corrección por "1 más"

    print(f"Índice de Moran I ({año_inicio}-{año_fin}): {I_observado:.4f}")
    print(f"p-valor (permutaciones={permutaciones}): {p_valor:.4f}")

    return I_observado, p_valor


In [18]:
calcular_moran_con_pvalor(muertes, poblacion_total, 2009, 2023)
calcular_moran_con_pvalor(muertes, poblacion_total, 2009, 2013)
calcular_moran_con_pvalor(muertes, poblacion_total, 2014, 2018)
calcular_moran_con_pvalor(muertes, poblacion_total, 2019, 2023)


Índice de Moran I (2009-2023): 0.1431
p-valor (permutaciones=999): 0.0010
Índice de Moran I (2009-2013): 0.0754
p-valor (permutaciones=999): 0.0010
Índice de Moran I (2014-2018): 0.1050
p-valor (permutaciones=999): 0.0010
Índice de Moran I (2019-2023): -0.0128
p-valor (permutaciones=999): 0.5000


(-0.012795026981602843, 0.5)